In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/working/__notebook__.ipynb


In [2]:
#prepare the label dict stuff
!wget https://raw.githubusercontent.com/physionetchallenges/physionetchallenges.github.io/master/2020/Dx_map.csv
labels = pd.read_csv('Dx_map.csv')
labels['index'] = range(len(labels))
label_dict = {}
f = open('label_dict','w+')
for count,out in enumerate(zip(labels['dx'],labels['SNOMED code'],labels['Abbreviation'],labels['index'])):
    label_dict[out[1]] = count
    print_str = '\t'.join([str(item) for item in out])
    f.write(print_str + '\n')
f.close()

--2020-06-14 15:27:03--  https://raw.githubusercontent.com/physionetchallenges/physionetchallenges.github.io/master/2020/Dx_map.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4207 (4.1K) [text/plain]
Saving to: ‘Dx_map.csv’

Dx_map.csv          100%[===================>]   4.11K  --.-KB/s    in 0s      

2020-06-14 15:27:04 (27.9 MB/s) - ‘Dx_map.csv’ saved [4207/4207]



In [3]:
import wfdb
import base64
import datetime
import gzip
label_count = {}
def prepare_data(dir_add, outfile):
    
    names = os.listdir(dir_add)
    names = [name[:-4] for name in names]
    names = list(set(names))

    f = gzip.open(outfile,'at+')
    for name in tqdm(names):
        file_add = dir_add + name
        data = wfdb.rdsamp(file_add)
        
        label = []
        for item in data[1]['comments'][2][4:].split(','):
            try:
                label.append(label_dict[int(item)])
            except:
                print('cannot found label ', item)
                label_dict[int(item)] = len(label_dict)
                label.append(label_dict[int(item)])
                print('label', item, 'added')
                
        if(len(label) == 0): continue
        #print(label)
        fea = str(data[1])
        data = np.asarray(data[0])
        onehot_label = [0] * len(label_dict)
        for label_item in label:
            onehot_label[label_item] = 1
            label_count[label_item] = label_count.get(label_item,0) + 1
        onehot_label = np.asarray(onehot_label)
        output_str = [name, base64.b64encode(data.astype(np.float16)).decode("utf-8"), base64.b64encode(onehot_label.astype(np.float16)).decode("utf-8"), fea]
        f.write('\t'.join(output_str) + '\n')
        #break

In [4]:
#1G
!wget -O PhysioNetChallenge2020_Training_CPSC.tar.gz \
https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/
!tar -zxf PhysioNetChallenge2020_Training_CPSC.tar.gz
!rm PhysioNetChallenge2020_Training_CPSC.tar.gz
print('prepared')

prepare_data('/kaggle/working/Training_WFDB/','phycom_data')
print(sum([1 for line in gzip.open('phycom_data','rt')]))
!rm -rf /kaggle/working/Training_WFDB/

--2020-06-14 15:27:04--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 859952525 (820M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’

PhysioNetChallenge2 100%[===================>] 820.11M  14.6MB/s    in 57s     

2020-06-14 15:28:02 (14.5 MB/s) - ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’ saved [859952525/859952525]

prepared



6877


In [5]:
#1g
!wget -O PhysioNetChallenge2020_Training_PTB.tar.gz \
https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_PTB.tar.gz/
!tar -zxf PhysioNetChallenge2020_Training_PTB.tar.gz
!rm PhysioNetChallenge2020_Training_PTB.tar.gz
print('prepared')

prepare_data('/kaggle/working/Training_PTB/','phycom_data')
print(sum([1 for line in gzip.open('phycom_data','rt')]))
!rm -rf /kaggle/working/Training_PTB/

--2020-06-14 15:31:30--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_PTB.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 932616724 (889M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_PTB.tar.gz’

PhysioNetChallenge2 100%[===================>] 889.41M  14.7MB/s    in 61s     

2020-06-14 15:32:32 (14.6 MB/s) - ‘PhysioNetChallenge2020_Training_PTB.tar.gz’ saved [932616724/932616724]

prepared


cannot found label  698247007
label 698247007 added
cannot found label  368009
label 368009 added
cannot found label  84114007
label 84114007 added
cannot found label  53741008
label 53741008 added
cannot found label  314208002
label 314208002 added
cannot found label  204384007
label 204384007 added
cannot found label  282825002
label 282825002 added

7393


In [6]:
#632MB
!wget -O PhysioNetChallenge2020_Training_E.tar.gz \
https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_E.tar.gz/
!tar -zxf PhysioNetChallenge2020_Training_E.tar.gz
!rm PhysioNetChallenge2020_Training_E.tar.gz
print('prepared')

prepare_data('/kaggle/working/WFDB/','phycom_data')
print(sum([1 for line in gzip.open('phycom_data','rt')]))
!rm -rf /kaggle/working/WFDB/

--2020-06-14 15:36:11--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_E.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 502442681 (479M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_E.tar.gz’

PhysioNetChallenge2 100%[===================>] 479.17M  14.7MB/s    in 34s     

2020-06-14 15:36:45 (14.3 MB/s) - ‘PhysioNetChallenge2020_Training_E.tar.gz’ saved [502442681/502442681]

prepared



17737


In [7]:
#0.6g
!wget -O PhysioNetChallenge2020_Training_StPetersburg.tar.gz \
https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_StPetersburg.tar.gz/
!tar -zxf PhysioNetChallenge2020_Training_StPetersburg.tar.gz
!rm PhysioNetChallenge2020_Training_StPetersburg.tar.gz
print('prepared')

prepare_data('/kaggle/working/Training_StPetersburg/','phycom_data')
print(sum([1 for line in gzip.open('phycom_data','rt')]))
!rm -rf /kaggle/working/Training_StPetersburg/

--2020-06-14 15:40:45--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_StPetersburg.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 591398198 (564M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_StPetersburg.tar.gz’

PhysioNetChallenge2 100%[===================>] 564.00M  15.0MB/s    in 39s     

2020-06-14 15:41:24 (14.6 MB/s) - ‘PhysioNetChallenge2020_Training_StPetersburg.tar.gz’ saved [591398198/591398198]

prepared


cannot found label  251182009
label 251182009 added
cannot found label  425856008
label 425856008 added
cannot found label  266257000
label 266257000 added
cannot found label  251173003
label 251173003 added
cannot found label  13640000
label 13640000 added
cannot found label  60423000
label 60423000 added
cannot found label  251168009
label 251168009 added
cannot found label  57054005
label 57054005 added
cannot found label  49260003
label 49260003 added

17811


In [8]:
#0.5g
!wget -O PhysioNetChallenge2020_Training_2.tar.gz \
https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_2.tar.gz/
!tar -zxf PhysioNetChallenge2020_Training_2.tar.gz
!rm PhysioNetChallenge2020_Training_2.tar.gz
print('prepared')

prepare_data('/kaggle/working/Training_2/','phycom_data')
print(sum([1 for line in gzip.open('phycom_data','rt')]))
!rm -rf /kaggle/working/Training_2/

--2020-06-14 15:44:22--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_2.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 423381508 (404M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_2.tar.gz’

PhysioNetChallenge2 100%[===================>] 403.77M  15.2MB/s    in 28s     

2020-06-14 15:44:51 (14.3 MB/s) - ‘PhysioNetChallenge2020_Training_2.tar.gz’ saved [423381508/423381508]

prepared



21264


In [9]:
f = open('label_dict_added','w+')
for key in label_dict.keys():
    f.write(str(key) + '\t' + str(label_dict[key]) + '\n')
f.close()

In [10]:
{k: v for k, v in sorted(label_count.items(), key=lambda item: item[1], reverse=True)}

{61: 3176,
 82: 2754,
 76: 2402,
 90: 2328,
 8: 1961,
 66: 1928,
 81: 1722,
 0: 1597,
 83: 1576,
 86: 1475,
 49: 1401,
 69: 1395,
 62: 1168,
 84: 968,
 46: 940,
 41: 905,
 44: 871,
 91: 818,
 67: 791,
 57: 760,
 47: 505,
 33: 493,
 80: 468,
 71: 465,
 35: 451,
 85: 424,
 95: 395,
 58: 384,
 51: 374,
 17: 288,
 6: 281,
 10: 241,
 60: 208,
 68: 188,
 42: 180,
 21: 161,
 24: 141,
 27: 140,
 19: 137,
 32: 128,
 77: 106,
 100: 89,
 75: 84,
 14: 79,
 43: 72,
 11: 62,
 7: 62,
 87: 58,
 12: 52,
 101: 46,
 1: 44,
 13: 43,
 9: 41,
 45: 40,
 98: 38,
 110: 37,
 88: 36,
 23: 35,
 48: 25,
 114: 23,
 4: 19,
 74: 18,
 107: 16,
 94: 16,
 115: 15,
 73: 14,
 103: 12,
 89: 12,
 72: 11,
 104: 9,
 38: 9,
 59: 9,
 106: 8,
 102: 8,
 109: 7,
 105: 7,
 78: 7,
 116: 7,
 118: 7,
 108: 6,
 40: 6,
 121: 6,
 29: 6,
 79: 5,
 15: 5,
 96: 4,
 64: 3,
 117: 3,
 55: 3,
 63: 3,
 111: 2,
 112: 2,
 113: 2,
 97: 2,
 5: 2,
 119: 2,
 16: 2,
 122: 2,
 39: 2,
 120: 1,
 36: 1,
 25: 1,
 93: 1,
 99: 1,
 26: 1,
 31: 1,
 20: 1,
 50: 1

In [11]:
sum([1 for line in gzip.open('phycom_data','rt')])

21264